In [1]:
import time

from model import Model, Optimizer, WavenetData

from IPython.display import Audio
from IPython.core.debugger import Tracer
from matplotlib import pyplot as plt
from matplotlib import pylab as pl
from IPython import display
import torch
import numpy as np
#%matplotlib inline
%matplotlib notebook

In [25]:
model = Model(num_time_samples=10000, 
              num_blocks=2, 
              num_layers=12, 
              num_hidden=128, 
              num_classes=256, 
              sampled_generation=False)

model.load_state_dict(torch.load("trained_state_bach_11025_12_layers"))

data = WavenetData('bach_11025.wav',
                   input_length=model.scope,
                   target_length=model.last_block_scope,
                   num_classes=model.num_classes)


current scope:  1
current scope:  3
current scope:  7
current scope:  15
current scope:  31
current scope:  63
current scope:  127
current scope:  255
current scope:  511
current scope:  1023
current scope:  2047
current scope:  4095
current scope:  4096
current scope:  4098
current scope:  4102
current scope:  4110
current scope:  4126
current scope:  4158
current scope:  4222
current scope:  4350
current scope:  4606
current scope:  5118
current scope:  6142
current scope:  8190
scope:  12286


In [26]:
start_tensor = data.get_minibatch([90000])[0].squeeze()
#start_tensor = torch.zeros((model.scope)) + 0.0

plt.ion()
plt.plot(start_tensor.numpy())
plt.ioff()

In [6]:
print('generate...')
tic = time.time()
generated = model.generate(start_data=start_tensor, num_generate=100)
toc = time.time()
print('Generating took {} seconds.'.format(toc-tic))

fig = plt.figure()
plt.plot(generated.data[-100:].numpy())

generate...


KeyboardInterrupt: 

In [4]:
print('generate...')
tic = time.time()
model.sample_generation = False
#[fast_generated, support_generated] = model.fast_generate(600, first_samples=torch.zeros((1)))
[fast_generated, support_generated] = model.fast_generate(600, first_samples=start_tensor)
toc = time.time()
print('Generating took {} seconds.'.format(toc-tic))

fig = plt.figure()
plt.plot(fast_generated)

generate...
Generating took 20.40074896812439 seconds.


In [27]:
sample_length = 22050

tic = time.time()
#[generated_sample, support_generated] = model.fast_generate(sample_length, first_samples=torch.zeros((1))+0.1)
[generated_sample, support_generated] = model.fast_generate(sample_length, first_samples=start_tensor)
toc = time.time()
print('Generating took {} seconds.'.format(toc-tic))

Generating took 111.3427357673645 seconds.


In [29]:
np_sample = np.asarray(generated_sample)
print("sample: ", np_sample)

from scipy.io import wavfile
wavfile.write('generated_bach_11025_sample_12_layers.wav', 11025, np_sample)

sample:  [ 0.1484375  0.125      0.0234375 ...,  0.2421875  0.140625  -0.0390625]


In [28]:
fig = plt.figure()
plt.plot(generated_sample[0:1000])

from IPython.display import Audio
Audio(generated_sample, rate=11025)